In [14]:
!wget 'https://raw.githubusercontent.com/asigalov61/tegridy-tools/main/tegridy-tools/GPT2RGA.py'

--2023-12-06 20:13:58--  https://raw.githubusercontent.com/asigalov61/tegridy-tools/main/tegridy-tools/GPT2RGA.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38843 (38K) [text/plain]
Saving to: ‘GPT2RGA.py.1’

GPT2RGA.py.1        100%[===================>]  37.93K  --.-KB/s    in 0.002s  

2023-12-06 20:13:58 (19.6 MB/s) - ‘GPT2RGA.py.1’ saved [38843/38843]



In [17]:
import os
from datetime import datetime
import secrets
import copy
import tqdm
from tqdm import auto

from GPT2RGA import *

from miditok import REMI, CPWord
from miditoolkit import MidiFile

import matplotlib.pyplot as plt

In [6]:
# Creating a multitrack tokenizer configuration, read the doc to explore other parameters
config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(config)

# Loads a midi, converts to tokens, and back to a MIDI
midi = MidiFile('midis/Q1__8v0MFBZoco_0.mid')
tokens = tokenizer(midi) 

In [8]:
# Tokenize a whole dataset and save it at Json files
midi_paths = list(Path("midis/").glob("*.mid"))
data_augmentation_offsets = [2, 1, 1]  # data augmentation on 2 pitch octaves, 1 velocity and 1 duration values
tokenizer.tokenize_midi_dataset(midi_paths, Path("token"),
                                data_augment_offsets=data_augmentation_offsets)

# Constructs the vocabulary with BPE, from the token files
tokenizer.learn_bpe(
    vocab_size=10000,
    tokens_paths=list(Path("path", "to", "tokens_noBPE").glob("**/*.json")),
    start_from_empty_voc=False,
)

# Saving our tokenizer, to retrieve it back later with the load_params method
tokenizer.save_params(Path("token/tokenizer.json"))

Tokenizing MIDIs (token):   0%|          | 0/1071 [00:00<?, ?it/s]

Performing data augmentation: 100%|██████████| 1071/1071 [00:05<00:00, 179.87it/s]
Loading token files: 0it [00:00, ?it/s]

In [8]:
with open('token/tokenizer.json', 'r') as json_file:
    data = json.load(json_file)

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 6.72kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 13.3MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 27.7MB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 300kB/s]
model.safetensors: 100%|██████████| 440M/440M [00:01<00:00, 237MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text_data = data["text_data"]

In [ ]:
token_tensor = tokenizer.encode(tokens, add_special_tokens=True)